In [ ]:
import configparser
import re
from bs4 import BeautifulSoup
from fyers_api import fyersModel
from fyers_api import accessToken
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from time import sleep
import urllib.parse as urlparse
import pandas as pd
import requests
import re
import datetime
import numpy as np
from pynse import *
nse=Nse()

In [ ]:
config = configparser.ConfigParser()
config.read('credentials.ini')
app_id = config['fyers']['app_id']
app_secret = config['fyers']['app_secret']
redirect_url = config['fyers']['redirect_url']
user_id = config['fyers']['user_id']
password = config['fyers']['password']
two_fa = config['fyers']['two_fa']

In [ ]:
def today_is_a_trading_holiday():
    response = requests.get('https://zerodha.com/marketintel/holiday-calendar/')
    soup = BeautifulSoup(response.text, 'html.parser')
    soup = soup.find_all(name='h4')[0]
    holiday = soup.find(string=re.compile("The next trading holiday"))
    next_holiday = '-'.join(re.findall('([0-3][0-9]) (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)',holiday)[0])
    next_holiday = next_holiday+'-'+str(datetime.datetime.now().year)
    next_holiday = datetime.datetime.strptime(next_holiday,'%d-%b-%Y').date()
    return next_holiday == datetime.datetime.today().date()

In [ ]:
class Fyers:
    
    def __init__(self,app_id,app_secret,redirect_url,user_id,password,two_fa):
        self.app_id = app_id
        self.app_secret = app_secret
        self.redirect_url = redirect_url
        self.user_id = user_id
        self.password = password
        self.two_fa = two_fa
        self.set_token()
        self.master_contract_nse_cm = self.master_contract_nse_cm()
        self.master_contract_nse_fo = self.master_contract_nse_fo()

    def get_session_url(self):
        self.session = accessToken.SessionModel(client_id=self.app_id,
                                           secret_key=self.app_secret,
                                           redirect_uri=self.redirect_url, 
                                           response_type="code",
                                           grant_type="authorization_code")
        self.url = self.session.generate_authcode()
        
    def login_driver(self):
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--disable-gpu')
        driver = webdriver.Chrome(executable_path='/home/yashraj/Desktop/chromedriver',options=options)
        driver.get(self.url)
        form = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//div[@class="container login-main-start"]')))
        driver.find_element_by_xpath("//input[@id='fyers_id']").send_keys(user_id)
        driver.find_element_by_xpath("//input[@id='password']").send_keys(password)
        driver.find_element_by_xpath("//input[@id='pancard']").send_keys(two_fa)
        driver.find_element_by_xpath("//button[@id='btn_id']").click()
        sleep(2)        
        return driver
    
    def get_auth_code(self,driver):
        current_url = driver.current_url
        driver.close()
        print(current_url)
        parsed = urlparse.urlparse(current_url)
        auth_code = urlparse.parse_qs(parsed.query)['auth_code'][0]
        return auth_code
        
    def set_token(self):
        self.get_session_url()
        driver = self.login_driver()
        auth_code = self.get_auth_code(driver)
        self.session.set_token(auth_code)
        response = self.session.generate_token()
        access_token = response['access_token']
        self.fyers = fyersModel.FyersModel(client_id=app_id, token=access_token,log_path="/home/yashraj/Desktop/apiV2")
    
    def master_contract_nse_cm(self):
        response = requests.get('http://public.fyers.in/sym_details/NSE_CM.csv')
        response_text = response.text
        response_text = response_text.split('\n')
        master_contract = pd.DataFrame(list(map(lambda x: x.split(','),response_text)))
        master_contract = master_contract.drop(10,axis=1)
        master_contract.columns = ['fyers_token','symbol_details','exchange_instrument_type','minimum_lot_size','tick_size','isin','trading_session','last_update_date','expiry_date','symbol_ticker','exchange','segment','scrip_code']
        master_contract = master_contract[:-1]
        return master_contract
    
    def master_contract_nse_fo(self):
        response = requests.get('http://public.fyers.in/sym_details/NSE_FO.csv')
        response_text = response.text
        response_text = response_text.split('\n')
        master_contract = pd.DataFrame(list(map(lambda x: x.split(','),response_text)))
        master_contract = master_contract.drop(10,axis=1)
        master_contract.columns = ['fyers_token','symbol_details','exchange_instrument_type','minimum_lot_size','tick_size','isin','trading_session','last_update_date','expiry_date','symbol_ticker','exchange','segment','scrip_code']
        master_contract = master_contract[:-1]

        def get_instrument(details):
            try:
                year, month, day, strike_price, instrument_type = re.findall(r'(21|22|23|24|25|26) (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) ([0-3][0-9]) ([0-9]{1,5}|[0-9]{1,5}.[0-9]{1,2}) (CE|PE)',details)[0]
            except:
                year, month, day, instrument_type = re.findall(r'(21|22|23) (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec) ([0-3][0-9]) (FUT)',details)[0]
                strike_price = None

            expiry_date = datetime.datetime.strptime((year+month+day),'%y%b%d').date()
            return expiry_date, strike_price, instrument_type

        master_contract['temp'] = np.vectorize(get_instrument, otypes=['O'])(master_contract['symbol_details'])
        master_contract['expiry'] = master_contract['temp'].apply(lambda x: x[0])
        master_contract['strike_price'] = master_contract['temp'].apply(lambda x: x[1])
        master_contract['instrument_type'] = master_contract['temp'].apply(lambda x: x[2])
        master_contract['expiry_date'] = master_contract['expiry_date'].apply(lambda x: datetime.datetime.fromtimestamp(int(x)).date())
        master_contract.sort_values('expiry_date',inplace=True)
        master_contract.reset_index(drop=True,inplace=True)
        master_contract['strike_price'] = master_contract['strike_price'].astype(float)
        return master_contract
    
    def get_instrument_details(self,symbol,strike_price=None,instrument_type='EQ',expiry_offset=0):
        if instrument_type == 'EQ':
            return self.master_contract_nse_cm[self.master_contract_nse_cm['scrip_code'] == symbol].iloc[0]
        elif (instrument_type == 'CE') | (instrument_type == 'PE'):
            return self.master_contract_nse_fo[(self.master_contract_nse_fo['strike_price'] == strike_price) & 
                                               (self.master_contract_nse_fo['scrip_code'] == symbol) & 
                                               (self.master_contract_nse_fo['instrument_type'] == instrument_type)].iloc[expiry_offset]
        elif (instrument_type == 'FUT'):
            return self.master_contract_nse_fo[(self.master_contract_nse_fo['scrip_code'] == symbol) & 
                                        (self.master_contract_nse_fo['instrument_type'] == instrument_type)].iloc[expiry_offset]
    
    def get_orderbook(self):
        orderbook = self.fyers.orderbook()
        if orderbook['code'] == 200:
            return orderbook['orderBook']
        else:
            print('Error Fetching Orderbook')
        
    def get_tradebook(self):
        tradebook = self.fyers.tradebook()
        if tradebook['code'] == 200:
            return tradebook['tradeBook']
        else:
            print('Error Fetching Tradebook')
        
    def get_net_positions(self):
        net_positions = self.fyers.net_positions()
        if net_positions['code'] == 200:
            return net_positions['netPositions']
        else:
            print('Error Fetching Net Positions')
    
    def place_order(self,symbol_ticker,trade_type,quantity,product_type='INTRADAY',order_type='MARKET',limit_price=0,stop_price=0):
        '''
        product_type = CNC, INTRADAY
        symbol_ticker: MCX:SILVERMIC20NOVFUT
        quantity = 100000
        trade_type = BUY or SELL
        
        limit_order: stop_price = 0, input(limit_price)
        market_order: limitPrice = 0, stop_price = 0
        stop_order_market: limit_price = 0, input(stop_price)
        stop_limit_order: input(limit_price), input(stop_price)
        '''
       
        if order_type == 'LIMIT':
            type = 1
            stop_price = 0

        elif order_type == 'MARKET':
            type = 2
            limit_price = 0
            stop_price = 0

        elif order_type == 'SLM':
            type = 3
            limit_price = 0

        elif order_type == 'SLL':
            type = 4

        if trade_type.upper() == 'BUY':
            side = 1
        elif trade_type.upper() == 'SELL':
            side = -1

        data = {
                "symbol":symbol_ticker,
                "qty":abs(quantity),
                "type":type,
                "side":side,
                "productType":product_type,
                "limitPrice":limit_price,
                "stopPrice":stop_price,
                "validity":"DAY",
                "disclosedQty":0,
                "offlineOrder":"False",
                "stopLoss":0,
                "takeProfit":0
                }

        order_id = self.fyers.place_order(data)
        return order_id
    
    def stop_loss_order(self,order_id,stop_loss_percentage,type=3,base=0.05):
        '''
        type=3 means stop_loss_order_market
        '''
        
        for order in self.get_orderbook():
            if order['id'] == order_id['id']:
                if order['message'] == "TRADE CONFIRMED":
                    side = order['side']
                    traded_price = order['tradedPrice']
                    filled_quantity = order['filledQty']
                    ticker = order['symbol']
                    product_type = order['productType']
        
        if side == -1:
            stop_loss_side = 1
            stop_loss_price = traded_price + (traded_price*stop_loss_percentage/100)
            
        elif side == 1:
            stop_loss_side = -1
            stop_loss_price = traded_price - (traded_price*stop_loss_percentage/100)
            
        stop_loss_price = round(base*round(round(stop_loss_price,2)/base),2)
        
        data = {
                "symbol":ticker,
                "qty":abs(filled_quantity),
                "type":type,
                "side":stop_loss_side,
                "productType":product_type,
                "limitPrice":0,
                "stopPrice":stop_loss_price,
                "validity":"DAY",
                "disclosedQty":0,
                "offlineOrder":"False",
                "stopLoss":0,
                "takeProfit":0
                }
        
        stop_loss_order_id = self.fyers.place_order(data)
        return stop_loss_order_id
    
    def cancel_order(self, order_id): 
        '''
        cancel order by order_id
        '''
        self.fyers.cancel_order({"id":order_id})
    
    def close_position(self,symbol_ticker): 
        '''
        close position by symbol
        '''
        fyers.exit_positions({"symbol": symbol_ticker})
    
    def close_all_open_positions(self):
        '''
        close all positions using the net positions function
        '''
        open_positions = []
        for position in self.get_net_positions():
            if abs(position['netQty']) > 0:
                open_positions.append(position['symbol'])
                
        for symbol_ticker in open_positions:
            self.close_position(symbol_ticker)

In [ ]:
while True:
    
    weekday_today = datetime.datetime.now().weekday()

    if (weekday_today != 5) & (weekday_today != 6):

        if today_is_a_trading_holiday() == False:   

            fyers = Fyers(app_id,app_secret,redirect_url,user_id,password,two_fa)

            symbol = 'BANKNIFTY'
            capital_requirement = 200000
            capital_deployed = 1000000

            if symbol == 'BANKNIFTY':
                lot_size = int(fyers.master_contract_nse_fo[fyers.master_contract_nse_fo['scrip_code'] == symbol]['minimum_lot_size'].iloc[0])
                base = 100
            elif symbol == 'NIFTY':
                lot_size = int(fyers.master_contract_nse_fo[fyers.master_contract_nse_fo['scrip_code'] == symbol]['minimum_lot_size'].iloc[0])
                base = 50

            quantity = int((capital_deployed/capital_requirement)*lot_size)

            while datetime.datetime.now().time() < datetime.time(9,24,55):
                sleep(1)

            quote = nse.get_quote('BANKNIFTY', segment=Segment.FUT)['lastPrice']
            atm = base*round(quote/base)

            ce_ticker = fyers.get_instrument_details(symbol,atm+100,'CE')['symbol_ticker']
            pe_ticker = fyers.get_instrument_details(symbol,atm-100,'PE')['symbol_ticker']

            ce_order_id = fyers.place_order(ce_ticker,'SELL',quantity)
            pe_order_id = fyers.place_order(pe_ticker,'SELL',quantity)
            ce_stop_loss_order_id = fyers.stop_loss_order(ce_order_id,25)
            pe_stop_loss_order_id = fyers.stop_loss_order(pe_order_id,25)

            while datetime.datetime.now().time() < datetime.time(15,19,45):
                sleep(1)

            stop_losses_ids = [ce_stop_loss_order_id,pe_stop_loss_order_id]
            for stop_loss_id in stop_losses_ids:
                try:
                    fyers.cancel_order(stop_loss_id)
                except:
                    pass

            fyers.close_all_open_positions()


    next_day_date = datetime.datetime.now().date() + datetime.timedelta(days=1)
    next_time = datetime.time(9,15)

    time_to_sleep = datetime.datetime.combine(next_day_date,next_time) - datetime.datetime.now()
    sleep(time_to_sleep.seconds)